#  MelodyMaster
## Notebook03 - song2vec & Multiclass
### Idan Kashani & Or Raphael Bidusa

# Shalom again!
Today we will examine another representation of the data - song2vec, based on word2vec representation.
We will use this representation with all sorts of different model, hoping to get a better result than last time with our simple knn.

In [160]:
import numpy as np
import pandas as pd
import ast
import warnings
from sklearn.model_selection import GridSearchCV
from collections import Counter
import seaborn as sns
import math
import torch
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
np.random.seed(42)

### Loading the dataset and extracting the vocabulary.

In [76]:
train_df = pd.read_csv('../train.csv')
print(f'train shape: {train_df.shape}')
train_df['lyrics_as_list'] = train_df['lyrics'].map(lambda l: set(ast.literal_eval(str(l)))) #Why set?
vocabulary = list(sorted(set.union(*train_df['lyrics_as_list'].tolist())))
train_df['lyrics_as_list'] = train_df['lyrics'].map(lambda l: ast.literal_eval(str(l)))
print(f"Vocabulary length is: {len(vocabulary)}")

train shape: (8496, 6)
Vocabulary length is: 62178


## Song2Vec
The Song2Vec representation (s2v from now) is based on the [word2vec](https://en.wikipedia.org/wiki/Word2vec) representation for words - a representation that relies on context.
Training a word2vec model in Hebrew is not an easy task - it requires en enormous datasets and a lot of computing power and therefore we shall use an already trained model.
We are using the [Ronshm](https://github.com/Ronshm/hebrew-word2vec) model, based on the entire Hebrew-Wikipedia site.
The tokenization of this model is a bit different - the model separates nouns, adjectives and verbs and marks them with "NN_", "VB_" and "JJ_", we have decided to remove these marks to match our tokenization.

In [77]:
with open('../words_list_w2v.txt',encoding='utf-8') as f:
    words = f.read().split('\n')
    # Removing the last word - an empty word
    words.pop()
vectors = np.load('../words_vectors_w2v.npy')

words = [w[3:] if len(w)>3 and w[:3] in ['NN_','VB_','JJ_'] else w for w in words]

The word `שוטה` for example can be all 3, a noun, a verb and an adjective and therefore has 3 entries.

In [92]:
print(f"The word שוטה has {words.count('שוטה')} entries")
from collections import defaultdict

def list_duplicates(seq):
    tally = defaultdict(list)
    for i,item in enumerate(seq):
        tally[item].append(i)
    return ((key,locs) for key,locs in tally.items()
            if len(locs)>1)

dups_indx = [dup for dup in sorted(list_duplicates(words))]
print(f"There are {len(dups_indx)} types with multiple entries")

The word שוטה has 3 entries
There are 4123 types with multiple entries


We will merge those together, using the mean for the 3 entries.

In [93]:
from collections import OrderedDict

words = list(OrderedDict.fromkeys(words))
vectors_without_dups = vectors.copy()
to_delete = []
for d in dups_indx:
    indices = sorted(d[1])
    vectors_without_dups[indices[0]] = np.mean(vectors[indices])
    to_delete = to_delete + indices[1:]
vectors = np.delete(vectors_without_dups, to_delete, 0)

Now we have a vocabulary of 340,419(!!) words, with corresponding 340,419 vectors.

In [95]:
print(len(words))
print(len(vectors))

340419
340419


Now we will create a way for to jump between the two representations, words and vectors.

In [97]:
w2v = dict(zip(words,vectors))
v2w_rep = dict(zip([tuple(v) for v in vectors],words))
def v2w(vector):
    return v2w_rep[tuple(vector)]

We have a dictionary for word->vector function and a method for the vector->word function.

In [98]:
ahavti_vec = w2v["ישראל"]
print(ahavti_vec)
print(v2w(ahavti_vec))

[ 2.285577 -2.40504  -0.247569  1.565358 -0.507429  0.421352  2.919839
  0.175767 -0.81715   4.027268 -4.426804 -0.335856  1.37266  -2.470423
 -0.637528 -0.625319 -0.387694 -0.377863 -2.823385  0.453176 -0.18862
 -1.135968  1.384955  1.184353 -1.846833 -1.811182  2.862987  0.043025
 -0.208302  0.839443  0.249148 -0.77285  -2.413144  1.783451  2.926803
 -3.261037  0.36823  -3.402674  0.538493 -1.489707 -1.453904 -1.631255
  2.344507 -1.803826 -0.740482  1.634151 -0.077221  0.617151  0.294225
  1.010752 -1.132112  0.357843 -1.024291 -1.490058  0.439202 -3.380302
  1.791306 -3.620549  0.068354 -0.148103 -1.801437  0.181857  3.369811
  4.313795  0.721022  0.792466  2.422217  2.32172   0.197274 -2.948433
  1.928675  0.277213  0.733989 -1.983691  2.088614  1.454925  2.900067
  3.587338  3.282094  0.349965 -2.046078 -1.244647 -0.144329  1.375046
  1.199163 -0.049973  0.327037 -2.425939  0.120854  2.901864  2.422603
 -0.95862   0.298806  1.730013 -1.094935 -0.845992  2.312083 -3.531979
  1.801

## The power of word2vec
With word2vec representation each word is represented by a 100d vector.
In order to demonstrate the meaning of closeness in this vector space we have two methods, one for finding the closest word in the vector space and one for finding the k-closest-vectors.

In [99]:
from scipy.spatial import cKDTree
tree = cKDTree(vectors)

def closest_word(vector):
    return words[tree.query(vector, k=1)[1]]

def k_closest_words(vector, k=5):
    distances, close_words = tree.query(vector, k=k)
    return [(words[w], d) for w, d in zip(list(close_words.astype(int)), distances)]

One interesting result is looking at the closest vectors for the representation of Germany as a vector.
We get a list of countries with both geographic and historical proximity like Austria, Switzerland and even Prussia.

In [101]:
k_closest_words(w2v["גרמניה"],k=10)

[('גרמניה', 0.0),
 ('אוסטריה', 9.436663490929567),
 ('שווייץ', 10.70911817768223),
 ('פרוסיה', 10.974586621357817),
 ('פולין', 11.275983929305372),
 ('הולנד', 11.324757442402465),
 ('צרפת', 11.344306222048795),
 ("צ'כוסלובקיה", 11.5207771481726),
 ('הונגריה', 11.54691985215105),
 ('רוסיה', 11.644340376521805)]

One very interesting test is the analogy test.
Using vector subtraction we can get the "vectorised difference" between two words - the relation between them.
For example, subtracting between "England" and "London" will give us a relation vector of "Capital City".
Adding this "relation vector" to a different word and then finding the closest vector to this sum can sometimes give us the corresponding analogy for this word.
Let's try to add the relation vector of "Capital City" to France, Belgium, Japan or Germany.

In [103]:
# x to y is like a to ?
def analogy(x,y,a):
    return w2v[y]-w2v[x]+w2v[a]

In [132]:
print(closest_word(analogy("אנגליה", "לונדון", "צרפת")))
print(closest_word(analogy("אנגליה", "לונדון", "בלגיה")))
print(closest_word(analogy("אנגליה", "לונדון", "יפן")))
print(closest_word(analogy("אנגליה", "לונדון", "גרמניה")))

פריז
בריסל
טוקיו
ברלין


The model can also find gendered relations:

In [153]:
print(closest_word(analogy("מלך", "מלכה", "נסיך")))
print(closest_word(analogy("מלך", "מלכה", "מושל")))
print(closest_word(analogy("מלך", "מלכה", "ביישן")))
print(closest_word(analogy("מלך", "מלכה", "מוזיקאי")))

נסיכה
מושלת
ביישנית
מוזיקאית


For song2vec representation we will try two pooling methods.
Mean-Pooling - taking the mean vector of all the words in the song.
Max-Pooling - for each entry in the 100d vector - taking the max of all words in the song.
For words which do not appear in the dataset - we will ignore them completely.

In [158]:
def song2vec_mean(song):
    return np.mean([w2v[w] for w in song if w in words],axis=0)

def song2vec_max(song):
    return np.max([w2v[w] for w in song if w in words], axis=0)

words_set = set(words)
train_df['song2vec_mean'] = train_df['lyrics_as_list'].map(lambda l: np.mean([w2v[w] for w in l if w in words_set],axis=0))
train_df = train_df.dropna()
train_df['song2vec_max'] = train_df['lyrics_as_list'].map(lambda l: np.max([w2v[w] for w in l if w in words_set],axis=0))

# MultiClass
We will examine the power of the new representation with all different kinds of models - but this time instead of one-vs-all we will try MultiClass - the true task of this project.
We will try different sets of learning algorithms, for each set of algorithms we will find the best model using CV.
We will once again use the f1 score for comparing between the models.

## kNN - revisited

In [173]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import StratifiedShuffleSplit

X = np.array([np.array(w) for w in train_df['song2vec_mean']])
y = np.array(train_df['artist'])
knn = KNeighborsClassifier(metric='cosine')
param_grid = {'n_neighbors': np.arange(1, 151, 15)}
cv = StratifiedShuffleSplit(n_splits=3, random_state=42)
knn_gscv = GridSearchCV(knn, param_grid, cv=cv,scoring="f1_micro",error_score="raise",return_train_score=True)
knn_mean = knn_gscv.fit(X, y)
print("kNN Mean Pooling:")
print(f"Best Params: {knn_mean.best_params_}")
print(f"Best f1 score for validation: {100*knn_mean.best_score_}")
print(f"The corresponding train f1 score: {100*knn_mean.cv_results_['mean_train_score'][knn_mean.best_index_]}")

kNN Mean Pooling:
Best Params: {'n_neighbors': 46}
Best f1 score for validation: 6.60377358490566
The corresponding train f1 score: 9.971584699453553


In [175]:
X = np.array([np.array(w) for w in train_df['song2vec_max']])
knn = KNeighborsClassifier(metric='cosine')
param_grid = {'n_neighbors': np.arange(1, 151, 15)}
cv = StratifiedShuffleSplit(n_splits=3, random_state=42)
knn_gscv = GridSearchCV(knn, param_grid, cv=cv,scoring="f1_micro",error_score="raise",return_train_score=True)
knn_max = knn_gscv.fit(X, y)
print("kNN Max Pooling:")
print(f"Best Params: {knn_max.best_params_}")
print(f"Best f1 score for validation: {100*knn_max.best_score_}")
print(f"The corresponding train f1 score: {100*knn_max.cv_results_['mean_train_score'][knn_max.best_index_]}")

kNN Max Pooling:
Best Params: {'n_neighbors': 91}
Best f1 score for validation: 5.031446540880503
The corresponding train f1 score: 7.042622950819672


In [202]:
from sklearn.svm import SVC
X = np.array([np.array(w) for w in train_df['song2vec_mean']])
C_range = np.logspace(-2, 10, 4)
gamma_range = np.logspace(-9, 3, 4)
param_grid = dict(gamma=gamma_range, C=C_range)
clf = SVC(kernel='rbf')
cv = StratifiedShuffleSplit(n_splits=3, random_state=43)
grid_search = GridSearchCV(clf, param_grid, cv=cv,scoring="f1_micro",error_score="raise",return_train_score=True,verbose=3)
svm_mean = grid_search.fit(X, y)
print("SVM Mean Pooling:")
print(f"Best Params: {svm_mean.best_params_}")
print(f"Best f1 score for validation: {100*svm_mean.best_score_}")
print(f"The corresponding train f1 score: {100*svm_mean.cv_results_['mean_train_score'][svm_mean.best_index_]}")

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV 1/3] END C=0.01, gamma=1e-09;, score=(train=0.041, test=0.041) total time=   6.9s
[CV 2/3] END C=0.01, gamma=1e-09;, score=(train=0.041, test=0.041) total time=   6.3s
[CV 3/3] END C=0.01, gamma=1e-09;, score=(train=0.041, test=0.041) total time=   6.7s
[CV 1/3] END C=0.01, gamma=1e-05;, score=(train=0.041, test=0.041) total time=   6.6s
[CV 2/3] END C=0.01, gamma=1e-05;, score=(train=0.041, test=0.041) total time=   6.7s
[CV 3/3] END C=0.01, gamma=1e-05;, score=(train=0.041, test=0.041) total time=   6.7s
[CV 1/3] END C=0.01, gamma=0.1;, score=(train=0.041, test=0.041) total time=   7.7s
[CV 2/3] END C=0.01, gamma=0.1;, score=(train=0.041, test=0.041) total time=   7.9s
[CV 3/3] END C=0.01, gamma=0.1;, score=(train=0.041, test=0.041) total time=   7.5s
[CV 1/3] END C=0.01, gamma=1000.0;, score=(train=0.041, test=0.041) total time=  15.6s


KeyboardInterrupt: 

[1.e-02 1.e+01 1.e+04 1.e+07 1.e+10]
